In [3]:
import pandas as pd
from datetime import timedelta
import scipy.stats as st
import math
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
df_ = pd.read_csv('amazon_review.csv')
df = df_.copy()

In [5]:
def quick_info(dataframe):
    print("--------- HEAD ---------")
    print(dataframe.head(10))
    print("--------- SHAPE ---------")
    print(dataframe.shape)
    print("--------- COLUMNS ---------")
    print(dataframe.columns)
    print("--------- DESCRIBE ---------")
    print(dataframe.describe().T)
    print("--------- INFO ---------")
    print(dataframe.info())
    print("--------- FREQUENCY ---------")
    print(dataframe.nunique())
    print("--------- ANY NULL VALUES ---------")
    print(dataframe.isnull().values.any())
    print("--------- SUM OF NULL VALUES ---------")
    print(dataframe.isnull().sum())

quick_info(df)

--------- HEAD ---------
       reviewerID        asin  \
0  A3SBTW3WS4IQSN  B007WTAJTO   
1  A18K1ODH1I2MVB  B007WTAJTO   
2  A2FII3I2MBMUIA  B007WTAJTO   
3   A3H99DFEG68SR  B007WTAJTO   
4  A375ZM4U047O79  B007WTAJTO   
5  A2IDCSC6NVONIZ  B007WTAJTO   
6  A26YHXZD5UFPVQ  B007WTAJTO   
7  A3CW0ZLUO5X2B1  B007WTAJTO   
8  A2CYJO155QP33S  B007WTAJTO   
9  A2S7XG3ZC4VGOQ  B007WTAJTO   

                                       reviewerName helpful  \
0                                               NaN  [0, 0]   
1                                              0mie  [0, 0]   
2                                               1K3  [0, 0]   
3                                               1m2  [0, 0]   
4                                      2&amp;1/2Men  [0, 0]   
5                                           2Cents!  [0, 0]   
6                                        2K1Toaster  [0, 0]   
7  35-year Technology Consumer "8-tracks to 802.11"  [0, 0]   
8                                         4e

NOTE: We don't need to drop NA values since reviewerName and reviewText don't effect our process

In [6]:
# Task 1_1
# Average rating of the product
def rating_average(dataframe):
    return dataframe['overall'].mean()

rating_average(df)

4.587589013224822

In [7]:
# Time-Based Weighted Average

# Task 1_2
df['reviewTime'] = df['reviewTime'].apply(pd.to_datetime)

In [8]:
df['reviewTime'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 4915 entries, 0 to 4914
Series name: reviewTime
Non-Null Count  Dtype         
--------------  -----         
4915 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 38.5 KB


In [9]:
current_date = df['reviewTime'].max()

In [10]:
analysis_date = current_date + timedelta(days=2)

In [11]:
df['timeDiff'] = (analysis_date - df['reviewTime']).dt.days

In [12]:
df['timeDiff'].sort_values(ascending=False)

3974    1065
502     1064
1761    1064
2881    1064
2382    1063
        ... 
3690       2
1688       2
4364       2
2603       2
2629       2
Name: timeDiff, Length: 4915, dtype: int64

In [13]:
q1, q2, q3 = df['timeDiff'].quantile(q=[.25, .5, .75])

In [14]:
def time_based_weighted_average(dataframe, w1=32, w2=28, w3=24, w4=16):
    return (dataframe.loc[df["timeDiff"] <= q1, "overall"].mean() * w1 / 100,
            dataframe.loc[(dataframe["timeDiff"] > q1) & (dataframe["timeDiff"] <= q2), "overall"].mean() * w2 / 100,
            dataframe.loc[(dataframe["timeDiff"] > q2) & (dataframe["timeDiff"] <= q3), "overall"].mean() * w3 / 100,
            dataframe.loc[(dataframe["timeDiff"] > q3), "overall"].mean() * w4 / 100)

value1, value2, value3, value4 = time_based_weighted_average(df)

In [15]:
# Task 1_3
print(f'Interval 1: {value1}, Interval 2: {value2}, Interval 3: {value3}, Interval 4: {value4}\n'
      f'Time Based Weighted Average: {sum(time_based_weighted_average(df))}')

Interval 1: 1.502653721682848, Interval 2: 1.2981193785772691, Interval 3: 1.097198697068404, Interval 4: 0.711400651465798
Time Based Weighted Average: 4.6093724487943195


When divided `timeDiff` column into quartiles, the first quartile value has a higher weighting value as it has a higher value than the others. Also, as can be seen above, this shows a parallel decrease in the weighted value with the quartiles order.

In [16]:
print(f'Rating Average => {rating_average(df)}\n'
      f'Time Based Weighted Average => {sum(time_based_weighted_average(df))}')

Rating Average => 4.587589013224822
Time Based Weighted Average => 4.6093724487943195


In [17]:
# 2nd Approach w/ qcut
# def time_weighted_average(dataframe, w1=32,w2=26,w3=24,w4=18):
#     dataframe['weights'] = pd.qcut(dataframe["day_delta"], [0, 0.25, 0.5, 0.75, 1], labels=[32, 26, 24, 18]).astype("int")
#     weighted_avg = dataframe.groupby("weights").agg({"overall": "mean"}).reset_index()
#     time_weighted_scores = weighted_avg["weights"] * weighted_avg["overall"] / 100
#     return time_weighted_scores.sum(), weighted_avg
#
# time_weighted_score, weighted_avg = time_weighted_average(df)
# print(time_weighted_score)
# weighted_avg

In [18]:
# Task 2_1

# Create a column which contains info about not helpful reviews
df['helpful_no'] = df['total_vote'] - df['helpful_yes']

In [19]:
# Task 2_2

#Up-Down Difference Score
def score_up_down_diff(up, down):
    return up - down

In [20]:
#Average Rating Score

def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [21]:
#Wilson Lower Bound (WLB) Score
def wilson_lower_bound(up, down, confidence=0.95):
    """
    Calculates Wilson Lower Bound Score

    - The lower limit of the confidence interval to be calculated for the Bernoulli parameter p is accepted as the WLB score.
    - The score to be calculated is used for product ranking.
    - Note:
    If the scores are between 1-5, 1-3 are marked as negative, 4-5 as positive and then scores can be adjusted to Bernoulli.
    This brings with it some problems. For this reason, it is necessary to make a bayesian average rating.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """

    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [22]:
df["score_pos_neg_diff"] = df.apply(lambda x: score_up_down_diff(x['helpful_yes'], x['helpful_no']), axis=1)

In [23]:
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x['helpful_yes'], x['helpful_no']), axis=1)

In [24]:
df['wilson_lower_bound'] = df.apply(lambda x: wilson_lower_bound(x['helpful_yes'], x['helpful_no']), axis=1)

In [25]:
df.loc[:, ["score_pos_neg_diff", 'score_average_rating', 'wilson_lower_bound']].sort_values(by=["score_pos_neg_diff", 'score_average_rating', 'wilson_lower_bound'],ascending=False)

,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,1884,0.966,0.958
4212,1442,0.926,0.912
3449,1351,0.949,0.937
317,349,0.853,0.819
3981,85,0.806,0.732
...,...,...,...
4391,-3,0.000,0.000
4574,-5,0.308,0.127
3198,-5,0.143,0.026
2751,-102,0.068,0.035


In [26]:
# Task 2_3
# Top 20 comments by WLB value
df.sort_values(by='wilson_lower_bound', ascending=False).head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,timeDiff,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,703,68,1884,0.966,0.958
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,804,77,1351,0.949,0.937
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,580,126,1442,0.926,0.912
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,1034,73,349,0.853,0.819
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.000,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,159,4,41,0.918,0.808
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.000,I own it,1393545600,2014-02-28,283,60,68,284,8,52,0.882,0.785
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.000,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,778,27,85,0.806,0.732
3807,AFGRMORWY2QNX,B007WTAJTO,R. Heisler,"[22, 25]",I bought this card to replace a lost 16 gig in...,3.000,"Good buy for the money but wait, I had an issue!",1361923200,2013-02-27,649,22,25,650,3,19,0.880,0.700
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.000,Awesome Card!,1339200000,2012-09-06,823,51,65,824,14,37,0.785,0.670
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.000,Designed incompatibility/Don't support SanDisk,1348272000,2012-09-22,807,82,109,808,27,55,0.752,0.664


Up-Down Difference Score degeri dusuk, Average Rating Score degeri yuksek olmasina ragmen bazi yorumlar ust siralarda yer almistir. Ayni sekilde Up-Down Difference Score degeri yuksek, Average Rating Score degeri dusuk olan yorumlarin da asagi satirlarda yer aldigini farkedebiliriz. Bunun nedeni ise, Birinci olay icin WBL oranlarinin digerlerine gore yuksek olmasi, ikinci olay icin ise WBL nin dusuk olmasidir. Dahasi Average Rating Score u full yani 1 olan satirlarda gormek mumkun fakat bunlarin Up-Down degerleri cok dusuktur. Bu 3 kiyaslamada bize en guvenilir sonucu donen WLB oldugu icin onu baz aldik ve siralamayi yapmis olduk.